# Univariate fine-mapping workflows

1. Using `susieR::susie()`
2. Using `susiF.alpha::susif`

In [ ]:
%preview ../images/Protocol_level_scheme - Fine-mapping.jpeg

## Univariate SuSiE

In [ ]:
nohup sos run /home/hs3163/GIT/xqtl-pipeline/pipeline/fine_mapping/SuSiE/SuSiE.ipynb uni_susie  \
        --phenotype_list /mnt/mfs/statgen/xqtl_workflow_testing/demo/phenotype_reformat/demo.processed_phenotype.per_gene.recipe  \
        --genotype_list /mnt/mfs/statgen/xqtl_workflow_testing/demo/genotype_reformmating/demo_per_gene_plink/plink_per_gene_list  \
        --region_list  /home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_region_long \
        --container /mnt/mfs/statgen/containers/twas_latest.sif \
        --wd ./ \
        --name "demo" &  

### SuSiE results post processing


In [ ]:
sos run pipeline/SuSiE_post_processing.ipynb susie_to_tsv \
    --cwd output/ADGWAS_finemapping_extracted/Bellenguez/ --rds_path `ls GWAS_Finemapping_Results/Bellenguez/ADGWAS2022*rds ` \
    --region-list ~/1300_hg38_EUR_LD_blocks_orig.tsv \
    --container containers/stephenslab.sif 

## Univariate SuSiF

In [ ]:
sos run pipeline/SuSiE.ipynb uni_fsusie \
    --genoFile /mnt/vast/hpc/csg/molecular_phenotype_calling/eqtl/output/dlpfc_tad_region_list_plink_files/ROSMAP_NIA_WGS.leftnorm.filtered.filtered.plink_files_list.txt  \
    --cwd output/f_susie_tad_haQTL_pos_check_pure_2/ --name haQTL \
    --region-list dlpfc_tad_list \
    --phenoFile phenotype_list \
    --covFile covar_list --prior mixture_normal_per_scale  --container containers/stephenslab.sif 

### SuSiE results post processing


In [ ]:
sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/f_susie_tad_haQTL_pos_check_pure_2 --rds_path `ls output/f_susie_tad_haQTL_pos_check_pure_2/cache/*rds ` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif 

sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/f_susie_tad_haQTL_pos_check_pure_3 --rds_path `ls output/f_susie_tad_haQTL_pos_check_pure_3/cache/*rds ` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif


sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/f_susie_tad_meQTL_pos_check_pure/ --rds_path `ls output/f_susie_tad_meQTL_pos_check_pure/cache/*rds ` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif

sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/pQTL/ --rds_path `ls /mnt/vast/hpc/csg/molecular_phenotype_calling/pqtl_finemapping/output/susie_per_ID_tad/cache/*rds` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif


sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/f_susie_tad_haQTL_limited/ --rds_path `ls output/f_susie_tad_haQTL_limited/cache/*rds` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif

sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/f_susie_tad_haQTL_pos_resume/ --rds_path `ls output/f_susie_tad_haQTL_pos_resume/cache/*rds` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif

## Plotting susie

### PIP land scape plot

In [ ]:
sos run pipeline/SuSiE_post_processing.ipynb susie_pip_landscape_plot \
    --cwd output/test/ --plot_list plot_recipe_668 --annot_tibble ~/Annotatr_builtin_annotation_tibble.tsv -s force --container containers/stephenslab.sif  &

sos run pipeline/SuSiE_post_processing.ipynb susie_pip_landscape_plot \
    --cwd output/test/ --plot_list plot_recipe_ADGWAS_uni --annot_tibble ~/Annotatr_builtin_annotation_tibble.tsv -s force --container containers/stephenslab.sif  &

### UpSetR plot 

In [ ]:
sos run pipeline/SuSiE_post_processing.ipynb susie_upsetR_plot \
    --cwd output/test/ --plot_list UpsetR_recipe -s force --container containers/stephenslab.sif &

## FSuSiE estimated effect plot

In [ ]:
sos run pipeline/SuSiE_post_processing.ipynb fsusie_extract_effect \
    --cwd output/test/ --rds_path /mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/output/f_susie_tad_meQTL_pos_2/cache/meQTL.yuqi_mqtl.tad668.uni_Fsusie.mixture_normal_per_scale.rds \
    --phenoFile /mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/minify_methy_BMIQ.bed.gz \
    --container containers/stephenslab.sif